# Coleta de Dados para Análise do WDO

Este notebook demonstra o processo de coleta de dados para análise do mini dólar (WDO).

In [ ]:
# Importações necessárias
import sys
sys.path.append('../')  # Adiciona diretório raiz ao path

from src.data.collect_market_data import fetch_wdo_data
from src.data.economic_indicators import EconomicDataCollector
from datetime import datetime, timedelta
import pandas as pd
import os
from dotenv import load_dotenv

# Carrega variáveis de ambiente
load_dotenv()

# Configura exibição do pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

## 1. Definir período de análise

In [ ]:
# Define período de análise
end_date = datetime.now()
start_date = end_date - timedelta(days=30)  # Começamos com 30 dias para teste

print(f'Período de análise: {start_date.date()} até {end_date.date()}')

## 2. Coletar dados do BCB

In [ ]:
# Cria instância do coletor
collector = EconomicDataCollector()

# Coleta dados do BCB
bcb_data = collector.collect_all_indicators(
    start_date=start_date.strftime('%Y-%m-%d'),
    end_date=end_date.strftime('%Y-%m-%d')
)

# Exibe dados coletados
for name, df in bcb_data.items():
    print(f'
{name}:')
    print(df.head())

## 3. Coletar dados do FRED

In [ ]:
# Pega API key do arquivo .env
fred_api_key = os.getenv('FRED_API_KEY')

if fred_api_key:
    # Coleta dados incluindo FRED
    all_data = collector.collect_all_indicators(
        start_date=start_date.strftime('%Y-%m-%d'),
        end_date=end_date.strftime('%Y-%m-%d'),
        fred_api_key=fred_api_key
    )
    
    # Filtra apenas dados do FRED
    fred_data = {k: v for k, v in all_data.items() if k.startswith('fred_')}
    
    for name, df in fred_data.items():
        print(f'
{name}:')
        print(df.head())
else:
    print('FRED API key não encontrada no arquivo .env')

## 4. Salvar dados coletados

In [ ]:
# Cria diretório se não existir
os.makedirs('../data/raw', exist_ok=True)

# Salva dados do BCB
for name, df in bcb_data.items():
    df.to_csv(f'../data/raw/{name}_data.csv')
    print(f'Dados salvos: {name}_data.csv')

# Salva dados do FRED se disponíveis
if fred_api_key:
    for name, df in fred_data.items():
        df.to_csv(f'../data/raw/{name}_data.csv')
        print(f'Dados salvos: {name}_data.csv')